<a href="https://colab.research.google.com/github/SherinBojappa/prompting/blob/main/opt_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# use huggingface bits and bytes to get 8 bit model
# https://huggingface.co/docs/transformers/perf_infer_gpu_one
# paper - https://arxiv.org/abs/2208.07339
# use tpu as the runtime type

In [2]:
#load latest version of bits and bytes from here
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55.8 MB 1.1 MB/s 


In [3]:
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 15.7 MB/s 
     |████████████████████████████████| 7.6 MB 63.7 MB/s 
     |████████████████████████████████| 175 kB 15.5 MB/s 


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov 15 21:30:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
name = "facebook/opt-13b"
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_8bit = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(name)

Downloading:   0%|          | 0.00/719 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.86G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.87G [00:00<?, ?B/s]

...

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA path found: /usr/local/cuda/lib64/libcudart.so
CUDA_SETUP: Detected CUDA version 112
CUDA_SETUP: Loading binary /usr/local/lib/python3.7/dist-packages/bitsandbytes/libbitsandbytes_cuda112.so...


Downloading:   0%|          | 0.00/721 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [12]:
def generate_from_model(model, tokenizer, max_new_tokens, text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda(), max_new_tokens=max_new_tokens)
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

In [16]:
# get response from opt, this is the initial bootstrapping code different for different datasets
text = 'How would you tell if hypothesis is enatiled by premise'

In [19]:
template_entailment = generate_from_model(model_8bit, tokenizer, 100, text)

In [20]:
template_entailment

'How would you tell if hypothesis is enatiled by premise?\n\nI have a hypothesis that I want to test. I have a premise that I want to test. I have a conclusion that I want to test.\n\nI have a hypothesis that I want to test. I have a premise that I want to test. I have a conclusion that I want to test.\n\nI have a hypothesis that I want to test. I have a premise that I want to test. I have a conclusion that I want to test.\n\nI'

In [42]:
text = 'What is entailment in natural language?'
template_entailment = generate_from_model(model_8bit, tokenizer, 50, text)


In [43]:
template_entailment

'What is entailment in natural language?\n\nThe term entailment is used in natural language to refer to the relationship between two sentences. The relationship is a logical one, and it is a relationship between two sentences that are logically related.\n\nThe term entailment is used in natural'

In [46]:
text = 'What is sentiment analysis in natural language?'
template_sentiment = generate_from_model(model_8bit, tokenizer, 50, text)


In [47]:
template_sentiment

'What is sentiment analysis in natural language?\n\nSentiment analysis is the process of identifying the emotional state of a text. It is a sub-field of natural language processing (NLP) that deals with the analysis of text.\n\nSentiment analysis is a sub-field of'

In [50]:
text = 'What is inference in natural language?'
template_inference = generate_from_model(model_8bit, tokenizer, 50, text)

In [51]:
template_inference

'What is inference in natural language?\n\nInference is the process of drawing conclusions from the evidence available. Inference is a process of reasoning that is used in many different areas of science and technology. Inference is also used in many areas of social science and humanities.\n\n'

In [53]:
text = 'What is coreference resolution in natural language?'
template_coref = generate_from_model(model_8bit, tokenizer, 50, text)

In [54]:
template_coref

'What is coreference resolution in natural language?\n\nCoreference resolution is the process of determining the correct reference in a sentence.\n\nCoreference resolution is the process of determining the correct reference in a sentence.\n\nCoreference resolution is the process of determining the correct reference in a sentence'

In [56]:
text = 'What is word sense disambiguation in natural language?'
template_wsd = generate_from_model(model_8bit, tokenizer, 50, text)

In [57]:
template_wsd

'What is word sense disambiguation in natural language?\n\nWord sense disambiguation is the process of determining the meaning of a word in a sentence. It is a process that is used in natural language processing.\n\nWord sense disambiguation is a process that is used in natural'

In [58]:
text = 'What is sentence completion in natural language?'
template_sent_comp = generate_from_model(model_8bit, tokenizer, 50, text)

In [59]:
template_sent_comp

'What is sentence completion in natural language?\n\nSentence completion is the process of completing a sentence by adding a word or phrase to it. Sentence completion is a natural language processing task.\n\nSentence completion is a natural language processing task.\n\nSentence completion is the'

In [61]:
text = 'What is paraphrase identification in natural language?'
template_paraphrase_id = generate_from_model(model_8bit, tokenizer, 50, text)

In [62]:
template_paraphrase_id

'What is paraphrase identification in natural language?\n\nParaphrase identification is the process of identifying a paraphrase of a text. Paraphrases are a type of paraphrase that is not a direct quotation of the original text. Paraphrases are often used to make a point'

In [63]:
text = 'What is summarization in natural language?'
template_summarize = generate_from_model(model_8bit, tokenizer, 50, text)

In [64]:
template_summarize

'What is summarization in natural language?\n\nSummarization is the process of extracting the main ideas from a text and presenting them in a more compact form. Summarization is a very important part of the process of information retrieval.\n\nThe main idea of summarization is to'

In [65]:
text = 'What is question answering in natural language?'
template_qa = generate_from_model(model_8bit, tokenizer, 50, text)

In [66]:
template_qa

'What is question answering in natural language?\n\nQuestion answering in natural language is the process of answering questions posed in natural language. The process of question answering in natural language is a subset of the process of question answering in machine language.\n\nQuestion answering in natural language is the process of'